# Unsupervised approach

In [2]:
# Import necessary packages
import numpy as np
import pandas as pd
import pickle as pkl
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

import random

In [3]:
# Set up directories
root = Path('/Volumes/becell/Lab Projects/ERCstG_HighMemory/Data/Marc/1_SOC/1_ProtocolControlsMales')
moseq_path = root / "MoSeq/2024_05_23-11_06_49/results"
deepof_conditions_path = root / "DeepOF/conditions.csv"

# Upload conditions
target_values = pd.read_csv(deepof_conditions_path)

In [4]:
# Get all syllables from each file
syllable_dict = {}
all_syllables = set()

for file in moseq_path.glob('*.csv'):
    syllables = pd.read_csv(file, usecols=['syllable'])['syllable'].values
    syllable_dict[file.stem] = syllables
    all_syllables.update(syllables)

all_syllables = sorted(list(all_syllables))

In [5]:
# Create transition dict by time bins
transition_dict = {}
num_bins = 6

for file, syllables in syllable_dict.items():
    
    # Create tag from file name
    tag = file.split('DLC')[0]
    transition_dict[tag] = {} # Initialize the transition dictionary for this file
    
    # Split the syllables list into "num_bins" lists
    syllable_bins = np.array_split(syllables, num_bins)

    for bin_idx, syllables_subset in enumerate(syllable_bins):
        transition_matrix = np.zeros((len(all_syllables), len(all_syllables)))

        # Iterate over each state's list of values and update the matrix
        for i in range(len(syllables_subset) - 1):
            if syllables_subset[i] in all_syllables and syllables_subset[i + 1] in all_syllables:
                transition_matrix[syllables_subset[i]][syllables_subset[i + 1]] += 1 # Row, Column

        # Silence diagonal
        np.fill_diagonal(transition_matrix, 0)
        
        # Normalize the transition matrix (make rows sum to 1)
        row_sums = transition_matrix.sum(axis=1, keepdims=True)
        transition_matrix = np.divide(transition_matrix, row_sums, where=row_sums != 0)
        transition_matrix = np.nan_to_num(transition_matrix)

        # Add the transition matrix to the dictionary
        transition_dict[tag][bin_idx] = transition_matrix

# Export dataframe

In [6]:
flattened_rows = []

for tag, bin_dict in transition_dict.items():
    for bin_idx, matrix in bin_dict.items():
        flat = matrix.flatten()
        row = {
            'id': tag,
            'time_bin': bin_idx
        }

        n_rows, n_cols = matrix.shape
        for i in range(n_rows):
            for j in range(n_cols):
                # Avoid self transitions
                if i == j:
                    continue
                row[f"syllable_{i}_{j}"] = matrix[i, j]
        flattened_rows.append(row)

df = pd.DataFrame(flattened_rows)

# Add a column indicating the cue acording to the target values
df['learning'] = df['id'].map(target_values.set_index('experiment_id')['learning'])

# Add a column indicating the group acording to the target values
df['group'] = df['id'].map(target_values.set_index('experiment_id')['group'])

# Save the DataFrame to a CSV file
df.to_csv(root / '/Users/mcanela/Desktop/Behavior paper review/moseq_dynamics_mean.csv', index=False)

## Contrasts

In [121]:
times_to_contrast = [2,3]
conditions = {
    'learning': 'mediated',
    'group': 'paired'
}

# Select ids from target_values based on conditions
filtered_ids = target_values.loc[
    (target_values['learning'] == conditions['learning']) &
    (target_values['group'] == conditions['group']),
    'experiment_id'
].tolist()

# Filter transition_dict based on target_values
filtered_transition_dict = {}
for file, transitions in transition_dict.items():
    if file in filtered_ids:
        transition_matrices_to_keep = {}
        for bin, matrix in transitions.items():
            if bin in times_to_contrast:
                transition_matrices_to_keep[bin] = matrix
        # Only keep the transition matrices that are in times_to_contrast
        filtered_transition_dict[file] = transition_matrices_to_keep

# Prepare the transition matrices for the two conditions
transition_matrices_A = {file: transitions[2] for file, transitions in filtered_transition_dict.items() if 2 in transitions}
transition_matrices_B = {file: transitions[3] for file, transitions in filtered_transition_dict.items() if 3 in transitions}

In [124]:
# Clean matrices of zeros
# Combine all matrices into one list
all_matrices = list(transition_matrices_A.values()) + list(transition_matrices_B.values())

# Stack into 3D array
stacked = np.stack(all_matrices)  # shape: (n_matrices, n_rows, n_cols)

# Find zero-only rows and cols across all matrices
rows_all_zero = np.all(np.all(stacked == 0, axis=0), axis=1)  # shape: (n_rows,)
cols_all_zero = np.all(np.all(stacked == 0, axis=0), axis=0)  # shape: (n_cols,)

# Get indices to keep
keep_rows = ~rows_all_zero
keep_cols = ~cols_all_zero

# Clean the dictionaries
def clean_dict(d):
    return {k: v[np.ix_(keep_rows, keep_cols)] for k, v in d.items()}

transition_matrices_A = clean_dict(transition_matrices_A)
transition_matrices_B = clean_dict(transition_matrices_B)

In [ ]:
# Distance between the two conditions
from scipy.special import rel_entr
def compute_distance(transmats_A, transmats_B, distance_func='manhattan'):

    # Create a list of transition matrices for each condition
    A = list(transmats_A.values())
    B = list(transmats_B.values())

    # Combine the matrices (axis=2 will stack them along the 3rd dimension)
    a = np.dstack(A)
    b = np.dstack(B)

    # Calculate the mean along the 3rd dimension, ignoring NaN values
    a_mean = np.nanmean(a, axis=2)
    b_mean = np.nanmean(b, axis=2)

    # Calculate the distance
    if distance_func == 'manhattan':
        distance = np.nansum(np.abs(a_mean - b_mean))
    elif distance_func == 'frobenius':
        distance = np.sqrt(np.nansum((a_mean - b_mean) ** 2))
    else:
        raise ValueError("Unsupported distance function")

    return distance

# Compute the global test statistic
Tobs = compute_distance(transition_matrices_A, transition_matrices_B)

# Apply the bootstrap
Tnull = []
random.seed(123)
for _ in range(5000):
    shuffled_A = {}
    shuffled_B = {}
    for id in filtered_ids:
        if random.random() < 0.5:
            shuffled_A[id] = transition_matrices_A[id]
            shuffled_B[id] = transition_matrices_B[id]
        else:
            shuffled_A[id] = transition_matrices_B[id]
            shuffled_B[id] = transition_matrices_A[id]

    Tnull.append(compute_distance(shuffled_A, shuffled_B))

# p_value = np.sum(Tnull > Tobs) / len(Tnull)

from scipy.special import erf
zscore = (Tobs - np.mean(Tnull)) / np.std(Tnull)
p_value = (1 - erf(zscore / np.sqrt(2))) / 2

print(f"Observed T statistic: {Tobs}")
print(f"Bootstrap p-value: {p_value}")
plt.hist(Tnull, bins=30);